# Database population Script

## Populating the DB

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ta.momentum
import yfinance as yf
import sklearn as sk
from ta.momentum import RSIIndicator

# Get the data for the top companies in the S&P 500
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table = data[0]
tickers = table['Symbol'].tolist()[:50]

# Download the data for each company
data_pds = {}
for ticker in tickers:
    try:
        data = yf.download(ticker)
        data['MA_10'] = data['Close'].rolling(10).mean()
        data['MA_50'] = data['Close'].rolling(50).mean()
        # rsi = RSIIndicator(data['Close'], window=14)
        # rsi = rsi.rsi()
        data_pds[ticker] = data.dropna()


    except:
        print(f"Error downloading {ticker}")
        continue






[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Train a linear regression model for each company
models = {}
for ticker, df in data_pds.items():
    X = df[['Close','MA_10', 'MA_50']] #use the rolling average
    y = df['Close'].shift(-1).dropna() #predict the next day's closing price
    X = X[:-1]

    # Check if there are enough samples to split
    if len(X) < 2:
        print(f"Not enough data for {ticker}")
        continue

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()
    model.fit(X_train, y_train)

    models[ticker] = model







Not enough data for AMTM


In [20]:
#Ask the user to input a stock to predict the price for, and the specific date

stock = 'AAPL'
date = '2020-03-19'

if stock in data_pds:
    df = data_pds[stock]
    X = df[['Close','MA_10', 'MA_50']]
    X = X[:-1]
    y = df['Close'].shift(-1).dropna()

    model = models[stock]
    prediction = model.predict([X.loc[date]])
    actual = y.loc[date]

    print(f"Predicted price for {stock} on {date}: {prediction[0]}\n")
    print(f"Actual price for {stock} on {date}: {actual}")



Predicted price for AAPL on 2020-03-19: [61.32029176]

Actual price for AAPL on 2020-03-19: Ticker
AAPL    57.310001
Name: 2020-03-19 00:00:00, dtype: float64
